```python
# Copyright 2022 Bloomberg Finance L.P.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

# Accounts Learning Item

### What is an account?

We have completed creating a class to represent a single position, but we often have multiple positions and want to organize and group them. This is what we call an account. An account is a logical group of positions that are related. For example, you may have an account specifically dedicated to your retirement goals. This account may hold positions and securities that will grow over time. You may also have an account for a trip you want to take next year which contains fixed income debt, which won't won't fluctuate rapidly in value. 

Below is an example of an account you could have.

I have an account named *"My Retirement Account"* which holds:

 ***2,000*** shares of ***MSFT USD***, 
 ***200*** shares of ***AAPL USD***, and
 ***100*** shares of ***TSLA USD***

### Problem Definition

We want to build a class that represents an account. We should be able to construct accounts with a set of positions and an account name. We should be able to get the name of the account from the object. 

We should be able to query with a set of security names/security objects (the list could contain both) and return a map of our input to the matching position object if present.
Additionally we should be able to add an arbitrary set of positions to our account, with duplicates being able to set existing positions. 

Lastly we should be able to remove accounts based on a set of incoming security names/security objects, any securities that aren't actually contained can be ignored.

- Allow for an account to be created with a set of positions and an account name
- Allow for querying of the current account's name
- Allow for the querying of all positions
- Allow for querying of positions with a set of security names/security objects
    - The set can contain both security name string and security objects
    - The return value should be a map with the query value to the position
    - The set can contain securities not present in the account. These should be ignored.
- Allow for positions to be added to the account with a set of position objects. Incoming positions should update existing positions
- Allow for the removal of positions from the account with a set of security names/security objects. Securities not in the account should be ignored.

### Provided Tools

#### *Data Source*

For this section no data generators are provided

#### *Solution Interface*

Your solution will need to follow the interface provided in the lab. Below is a snippet of the interface for securities that you can inherit from. The methods that will be tested are displayed & will need to be overwritten with your implementation. You're free to add more methods then what is displayed in the interface!

```python
#filename interfaces.accountInterface.py
#Account Class Interface

from positionInterface import positionInterface
from typing import Any, Dict, Set, Iterable
class accountInterface():
    def __init__(self, positions: Set[positionInterface], accountName: str) -> None:
        pass

    def getName(self) -> str:
        pass

    def getAllPositions(self) -> Iterable[positionInterface]:
        pass

    def getPositions(self, securities: Set) -> Dict[Any, positionInterface]:
        pass

    def addPositions(self, positions: Set[positionInterface]) -> None:
        pass
    
    def removePositions(self, securities: Set) -> None:
        pass
```

To successfully import the interface into your solution cell you'll need to add the below code snippet. Due to the filesystem layout in the current Jupyter notebook, importing relative .pys requires us to adjust the system path.

```python
import os
import sys
module_path = os.path.abspath('..')
if module_path not in sys.path:
    sys.path.append(module_path)
```

#### *Testing*

Once you have completed & saved your solution you can run the test file to validate that your solution works as expected. For the test to run the following need to be true.
- Saved code to file **implementations/accountSolution.py**
- Create a class with the name **account** that inherits from **accountInterface**

### Stretch Goals

If you complete your class & have a solution with valid tests try completing the following stretch goals 

- Update your class init to handle the position set being optional
- Add custom implementation for class's __str__ method. The account print method shouldn't reimplement the positions's print str method but should utilize it.
- Add a method to remove all positions from an account.
- Develop tests for the new methods created

In [ ]:
#%%writefile ../implementations/accountSolution.py 
#Uncomment line above & run cell to save solution
#TODO Define class that implements accountInterface & allows for the management of an account

In [ ]:
#%conda install ipytest
#%pip install ipytest
import ipytest

ipytest.autoconfig()

In [ ]:
%%ipytest -qq
from implementations.securitySolution import security
from implementations.positionSolution import position
import implementations.accountSolution
import importlib
importlib.reload(implementations.accountSolution)

def test_getAccountName():
    #GIVEN
    EXPECTED_NAME = "MY TEST ACCOUNT"
    EXPECTED_POSITIONS = set()

    #WHEN 
    testObj = implementations.accountSolution.account(EXPECTED_POSITIONS, EXPECTED_NAME)

    #EXPECT 
    assert(testObj.getName() == EXPECTED_NAME)

def test_getAllPositions():
    #GIVEN
    EXPECTED_NAME = "MY TEST ACCOUNT"
    EXPECTED_POSITIONS = set()
    EXPECTED_POSITIONS.add(position("TEST_SEC_A", 1000))
    EXPECTED_POSITIONS.add(position("TEST_SEC_B", 2000))

    #WHEN 
    testObj = implementations.accountSolution.account(EXPECTED_POSITIONS, EXPECTED_NAME)
    returnPosItr = testObj.getAllPositions()

    #EXPECT
    assert(len(returnPosItr) == len(EXPECTED_POSITIONS))

    for item in list(returnPosItr):
        assert(item in EXPECTED_POSITIONS)
        EXPECTED_POSITIONS.remove(item)
        returnPosItr.remove(item)

    assert(len(returnPosItr) == 0)
    assert(len(EXPECTED_POSITIONS) == 0)

def test_getPositions():
    #GIVEN
    EXPECTED_NAME = "MY TEST ACCOUNT"
    EXPECTED_POSITIONS = set()
    EXPECTED_POSITIONS.add(position("TEST_SEC_A", 1000))
    EXPECTED_POSITIONS.add(position("TEST_SEC_B", 2000))
    KEY_LIST = [security("TEST_SEC_A"), "TEST_SEC_B", "TEST_NOT_FOUND_STR", security("TEST_NOT_FOUND_POS")]
    EXPECTED_MAP = {
        KEY_LIST[0] : position("TEST_SEC_A", 1000), 
        KEY_LIST[1] : position("TEST_SEC_B", 2000),
    }

    #WHEN 
    testObj = implementations.accountSolution.account(EXPECTED_POSITIONS, EXPECTED_NAME)
    returnPosItr = testObj.getPositions(KEY_LIST)

    #EXPECT
    assert(len(returnPosItr) == len(KEY_LIST) - 2)
    for item in KEY_LIST:
        if isinstance(item, security) and "NOT_FOUND" in item.getName():
            assert(item not in returnPosItr)
        elif isinstance(item, str) and "NOT_FOUND" in item:
            assert(item not in returnPosItr)
        else:
            assert(returnPosItr[item].getSecurity().getName() == EXPECTED_MAP[item].getSecurity().getName())
            assert(returnPosItr[item].getPosition() == EXPECTED_MAP[item].getPosition())
            
def test_addPositions():
    EXPECTED_NAME = "MY TEST ACCOUNT"
    START_POSITIONS = set([position("TEST_SEC_A", 1000), position("TEST_SEC_B", 2000)])
    UPDATE_POSITIONS = set([position("TEST_SEC_B", 3000), position("TEST_SEC_C", 1500)])
    EXPECTED_POSITIONS = {
        "TEST_SEC_A" : 1000,
        "TEST_SEC_B" : 3000,
        "TEST_SEC_C" : 1500
    }

    #WHEN 
    testObj = implementations.accountSolution.account(START_POSITIONS, EXPECTED_NAME)
    testObj.addPositions(UPDATE_POSITIONS)
    returnPosItr = testObj.getAllPositions()

    #EXPECT
    assert(len(returnPosItr) == len(EXPECTED_POSITIONS))

    for item in list(returnPosItr):
        assert(item.getSecurity().getName() in EXPECTED_POSITIONS)
        assert(item.getPosition() ==  EXPECTED_POSITIONS[item.getSecurity().getName()])
        del EXPECTED_POSITIONS[item.getSecurity().getName()]
        returnPosItr.remove(item)

    assert(len(returnPosItr) == 0)
    assert(len(EXPECTED_POSITIONS) == 0)

def test_removePositions():
    EXPECTED_NAME = "MY TEST ACCOUNT"
    START_POSITIONS = set([position("TEST_SEC_A", 1000), position("TEST_SEC_B", 2000), position("TEST_SEC_C", 1500), position("TEST_SEC_D", 3500)])
    REMOVE_POSITIONS = set([security("TEST_SEC_B"), security("TEST_SEC_C")])
    EXPECTED_POSITIONS = {
        "TEST_SEC_A" : 1000,
        "TEST_SEC_D" : 3500
    }

    #WHEN 
    testObj = implementations.accountSolution.account(START_POSITIONS, EXPECTED_NAME)
    testObj.removePositions(REMOVE_POSITIONS)
    returnPosItr = testObj.getAllPositions()

    #EXPECT
    assert(len(returnPosItr) == len(EXPECTED_POSITIONS))

    for item in list(returnPosItr):
        assert(item.getSecurity().getName() in EXPECTED_POSITIONS)
        assert(item.getPosition() ==  EXPECTED_POSITIONS[item.getSecurity().getName()])
        del EXPECTED_POSITIONS[item.getSecurity().getName()]
        returnPosItr.remove(item)

    assert(len(returnPosItr) == 0)
    assert(len(EXPECTED_POSITIONS) == 0)